## 🔹 Configuración inicial (resumen)

- `%run templates/config.py` y `%run templates/models.py`: carga configuraciones y modelos externos.  
- `warnings.filterwarnings(...)`: oculta avisos de **openpyxl** y medias en arrays vacíos.  
- `comparison_date = '2025-09-01'`: fecha base para comparaciones temporales.  
- `ticket_cols`: mapea columnas clave entre **incidents**, **problems** y **problem_tasks**.  
- `time_cols_pairs`: define pares de fechas para medir duraciones (CREATED→RESOLVED, OUTAGE_BEGIN→CLOSED, etc.).  
- `patterns`: regex para identificar IDs de tickets (PRB, INC, PTASK, CHG).  
- `trace_changes`: diccionario para identificar los cambios en las tablas.  
- Comentarios: *Aztech España* (`aztecil/aztecsl`) y auditoría IberoLatam (ES, PT, CO, BR, AR, MX).  
- `logger.info('START')`: marca inicio de ejecución y transformaciones en logs.  


In [ ]:
# %pip install numba

In [ ]:
%run templates/config.py
%run templates/models.py

import warnings
# Suppress specific warning from openpyxl
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.styles.stylesheet")
warnings.filterwarnings("ignore", message="Mean of empty slice")

comparison_date = '2025-09-01'

ticket_cols = {
    'df_incidents':('NUMBER','PROBLEM'),
    'df_problem':('NUMBER','CREATED_OUT_OF_INCIDENT'),
    'df_problem_tasks':('NUMBER','PROBLEM'),
}

time_cols_pairs = {
    'df_incidents':[('CREATED','RESOLVED'),('CREATED','CLOSED'),('RESOLVED','CLOSED'),('OUTAGE_BEGIN','OUTAGE_END'),('OUTAGE_BEGIN','RESOLVED'),('OUTAGE_BEGIN','CLOSED')],
    'df_problem':[('CREATED','RESOLVED'),('CREATED','CLOSED'),('RESOLVED','CLOSED'),('OUTAGE_BEGIN','OUTAGE_END'),('OUTAGE_BEGIN','RESOLVED'),('OUTAGE_BEGIN','CLOSED')],
    'df_problem_tasks':[('CREATED','RESOLVED'),('CREATED','CLOSED'),('RESOLVED','CLOSED')],
}

patterns = [
    r'PRB\d{7}',   # Matches 'PRB' followed by 7 digits
    r'INC\d{8}',   # Matches 'INC' followed by 8 digits
    r'PTASK\d{7}', # Matches 'PTASK' followed by 7 digits
    r'CHG\d{7}'    # Matches 'CHG' followed by 7 digits
]

trace_changes = {}
casos_audit_descrip = {} # Dictionary to hold descriptions of audit cases

logger.info('START')
logger_transform.info('START')

# Test

In [ ]:
# ============================================================
# PREPARAR FICHEROS
# ============================================================

import fastparquet
from typing import Optional

def load_parquet_folder(folder_path: str, verbose: bool = True) -> pd.DataFrame:
    """
    Reads and merges all Parquet files from a folder into a single DataFrame.
    
    Args:
        folder_path (str): Path to the folder containing .parquet files.
        verbose (bool): If True, prints progress messages.
        
    Returns:
        pd.DataFrame: Merged DataFrame from all Parquet files (empty if none found).
    """
    if not os.path.exists(folder_path):
        if verbose:
            print(f"⚠️ Folder not found: {folder_path}")
        return pd.DataFrame()

    parquet_files = [
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if f.endswith(".parquet")
    ]

    if not parquet_files:
        if verbose:
            print(f"📂 No parquet files found in: {folder_path}")
        return pd.DataFrame()

    dfs = []
    for f in parquet_files:
        try:
            if verbose:
                print(f"📥 Reading: {os.path.basename(f)}")
            dfs.append(pd.read_parquet(f, engine='fastparquet'))
        except Exception as e:
            print(f"❌ Error reading {f}: {e}")

    if not dfs:
        return pd.DataFrame()

    merged_df = pd.concat(dfs, ignore_index=True, sort=False)
    if verbose:
        print(f"✅ Merged {len(dfs)} parquet files — shape: {merged_df.shape}")
    return merged_df

folder = "CRE_INC"
df_created_incidents = load_parquet_folder(folder)

folder = "DATA_inc"
df_incidents = load_parquet_folder(folder)

In [ ]:
df_created_incidents = df_created_incidents.pipe(clean_headers)
df_incidents = df_incidents.pipe(clean_headers)
print(df_incidents.shape)
df_incidents = df_incidents.drop_duplicates()
print(df_incidents.shape)

In [ ]:
mask = df_incidents['NUMBER'].isin(df_created_incidents['NUMBER'].unique())
df_incidents = df_incidents[mask]

In [ ]:
# df_incidents[[col for col in df_incidents.columns if not col.startswith("_")]]

df_incidents = (df_incidents
                .pipe(sort_values_of_incidents)
                .pipe(process_not_closed_business_optimized, time_cols_pairs['df_incidents'])
                .pipe(exceed_bands_priority_CRE_RES, 'CURRENT_PRIORITY')
                .pipe(marks)
                .pipe(sort_values_of_incidents, '_SIZE')
                )

In [ ]:
# Comprobamos: ningun ticket tiene mas de una company
# df_incidents.groupby('NUMBER')['COMPANY'].nunique().reset_index().pipe(lambda x: x[x['COMPANY']>1])

to_AZS = [
    "ALLIANZSEGUROS SA", 
    "ALLIANZ SEGUROS SA", 
    "ALLIANZ COMPANIA DE SEGUROS Y REASEGUROS SA"
]

df_incidents['COMPANY'] = (df_incidents['COMPANY']
                           .apply(standardize_company_name)
                           # .value_counts(dropna=False)
                           )

df_incidents['COMPANY'] = (df_incidents['COMPANY']
                           .replace(to_AZS, 'ALLIANZ SEGUROS SA')
                           )

# 'ALLIANZ COLOMBIA', 'ALLIANZ TECHNOLOGY GDN INTERNAL', 'ALLIANZ TECHNOLOGY EXTERNAL', 
# 'ALLIANZ DIRECT', 'ALLIANZ TECHNOLOGY', 'ALLIANZ TECHNOLOGY IBEROLATAM BRANCH', 'ALLIANZ BRAZIL EXTERNAL'

company = [
    'ALLIANZ SEGUROS SA',
    'ALLIANZ SPAIN EXTERNAL',
    'BBVA ALLIANZ EXTERNAL',
    'ALLIANZ TECHNOLOGY SL',
    'ALLIANZ SOLUCIONES DE INVERSION AGV',
    'BBVA',
    'ALLIANZ TECHNOLOGY IBEROLATAM EXTERNAL',
]

mask = df_incidents['COMPANY'].isin(company)
df_incidents = df_incidents[mask]
print(df_incidents.shape)

In [ ]:
df_incidents['_IS_Parent'] = df_incidents['PARENT_INCIDENT'].fillna('') == '' # remove childs
df_incidents['_IS_FCR'] = df_incidents['FCR'] != True

filters = [
    "_IS_Parent",
    "_IS_FCR",
]
df_incidents = apply_filters(df_incidents, filters)
print(df_incidents.shape)

In [ ]:
df_incidents.select_dtypes(include=['datetime'])

In [ ]:
df_incidents = df_incidents.pipe(create_dt_time_wvariants, cols=['CREATED','RESOLVED','CLOSED'])
print(df_incidents.shape)

## 🔹 Carga y depuración de datos

- `load_and_process_data(...)`: carga los dataframes principales:
  -  `df_incidents`, `df_problem`, `df_problem_tasks`.  
  -  `df_created_incidents`, `df_resolved_incidents`, `df_problems_pbi`.
- `original_columns`: diccionario con las columnas iniciales de cada dataframe (para referencia y trazabilidad).  
- `original_shapes`: diccionario con las dimensiones iniciales de cada dataframe (filas, columnas).  
- `handle_duplicates(df_incidents, 'df_incidents')`: elimina duplicados en la tabla **df_incidents**.  


In [ ]:
# excel_path = os.path.join(raw_data, "Data_INC.xlsx")
# df_origen = pd.read_excel(excel_path)
# df_origen[df_origen["Number"]=="INC29720425"]

In [ ]:
# tabulate_df(analyze_dataframe_columns_export(df), None)

In [ ]:
log_separator(
    title="Section: [MAIN LOAD DATA] Description: [Loading and processing main data]"
)

load_from_pickle = True

(
    df_created_incidents,
    df_resolved_incidents,
    df_problems_pbi,
    df_problem_tasks,
    df_problem,
    # df_incidents,
) = load_and_process_data(load_from_pickle=load_from_pickle)

original_columns = {
    # SECUNDARY (PowerBI)
    "df_created_incidents": df_created_incidents.columns.tolist(),
    "df_resolved_incidents": df_resolved_incidents.columns.tolist(),
    "df_problems_pbi": df_problems_pbi.columns.tolist(),
    # PRINCIPAL (ServiceNow)
    "df_incidents": df_incidents.columns.tolist(),
    "df_problem": df_problem.columns.tolist(),
    "df_problem_tasks": df_problem_tasks.columns.tolist(),
}
original_shapes = {
    # SECUNDARY (PowerBI)
    "df_created_incidents": df_created_incidents.shape,
    "df_resolved_incidents": df_resolved_incidents.shape,
    "df_problems_pbi": df_problems_pbi.shape,
    # PRINCIPAL (ServiceNow)
    "df_incidents": df_incidents.shape,
    "df_problem": df_problem.shape,
    "df_problem_tasks": df_problem_tasks.shape,
}

In [ ]:
trace_changes['1_Carga'] = print_shapes()

In [ ]:
log_separator(title="Section: [MAIN DROP DUPLICATES] Description: [Drop duplicate rows]")
log_separator("TABLE: 'df_incidents'", 3)
df_incidents = handle_duplicates(df_incidents, 'df_incidents').pipe(
    sort_values_of_incidents
)

In [ ]:
trace_changes['2_INC_Duplicados'] = print_shapes()

## 🔹 Relaciones entre dataframes y validación

- `dataframes_relationships`: metadatos con origen (**ServiceNow/PowerBI**), clave primaria (`pk`) y clave foránea (`fk`) de cada tabla.  
- `dataframes`: diccionario con los dataframes cargados (incidents, problems, tasks, created/resolved incidents, problems PBI).  
  - Nota: `df_incidents_grouped` se generará más adelante (último UPDATE por ticket).  
- `create_structured_relationship_map(...)`: construye un mapa estructurado de relaciones entre tablas.  
- `get_flat_relationships(...)`: aplana el mapa en una lista de relaciones simples.  
- `print_valid_relationships(...)`: muestra las relaciones válidas identificadas.  
- `create_table_comparisons(...)`: genera comparaciones de datos entre tablas relacionadas.  
- `try/except del dataframes`: libera memoria eliminando el diccionario si existe.  
- Salida final:  
  - Total de relaciones válidas encontradas.  
  - Número de relaciones con coincidencias de datos.  


In [ ]:
# metadata
dataframes_relationships = {
    'df_incidents': {'db': 'ServiceNow', 'pk': 'NUMBER', 'fk': 'PROBLEM'},
    'df_problem': {'db': 'ServiceNow', 'pk': 'NUMBER', 'fk': 'CREATED_OUT_OF_INCIDENT'},
    'df_problem_tasks': {'db': 'ServiceNow', 'pk': 'NUMBER', 'fk': 'PROBLEM'},
    'df_created_incidents': {'db': 'PowerBI', 'pk': 'NUMBER', 'fk': 'PROBLEM'},
    'df_resolved_incidents': {'db': 'PowerBI', 'pk': 'NUMBER', 'fk': 'PROBLEM'},
    'df_problems_pbi': {'db': 'PowerBI','pk': 'NUMBER','fk': 'CREATED_OUT_OF_INCIDENT'},
    'df_incidents_grouped': {'db': 'ServiceNow', 'pk': 'NUMBER', 'fk': 'PROBLEM'},
}

dataframes = {
    'df_incidents': df_incidents,
    'df_problem': df_problem,
    'df_problem_tasks': df_problem_tasks,
    'df_created_incidents': df_created_incidents,
    'df_resolved_incidents': df_resolved_incidents,
    'df_problems_pbi': df_problems_pbi,
}

# relationship map
relationship_map = create_structured_relationship_map(dataframes_relationships)

# relationships
flat_relationships = get_flat_relationships(relationship_map)
print(f"\nTotal semantically valid relationships identified: {len(flat_relationships)}")
print_valid_relationships(flat_relationships)

# comparison table
table_comparisons = create_table_comparisons(flat_relationships, dataframes)
casos_audit['PBI_vs_ServiceNow'] = table_comparisons
casos_audit_descrip['Relaciones con datos que coinciden'] = "Relaciones entre tablas donde las claves foráneas y primarias tienen coincidencias en los datos."
# print(f"Relationships with data matches: {len(table_comparisons)}")

try:
    del table_comparisons
    del dataframes
except NameError:
    pass

## 🔹 Enriquecimiento de columnas de fecha

- **Función `create_dt_time_wvariants(df)`**  
  - Crea variantes adicionales para cada columna `datetime` en el dataframe (excepto `UPDATED`, `OUTAGE_BEGIN`, `OUTAGE_END`).  
  - Genera nuevas columnas:
    - `_{col}_year`: año.  
    - `_{col}_month`: mes numérico.  
    - `_{col}_period`: periodo en formato mensual (`YYYY-MM`).  
    - `_{col}_day_name`: nombre del día de la semana.  
    - `_{col}_isnull`: indicador de valores nulos.  
    - `_{col}_str`: fecha en formato string (`YYYY-MM-DD HH:MM`).  

  - Se enriquece con variantes de tiempo cada dataset principal:  
    - `df_incidents`, `df_problem`, `df_problem_tasks`  
    - `df_created_incidents`, `df_resolved_incidents`, `df_problems_pbi`  


In [ ]:
df_incidents = df_incidents.pipe(create_dt_time_wvariants)
df_problem = df_problem.pipe(create_dt_time_wvariants)
df_problem_tasks = df_problem_tasks.pipe(create_dt_time_wvariants)

df_created_incidents = df_created_incidents.pipe(create_dt_time_wvariants)
df_resolved_incidents = df_resolved_incidents.pipe(create_dt_time_wvariants)
df_problems_pbi = df_problems_pbi.pipe(create_dt_time_wvariants)

In [ ]:
trace_changes['3_ColumnasFechas'] = print_shapes()

In [ ]:
# clean columns
df_incidents['COMPANY'] = df_incidents['COMPANY'].apply(standardize_company_name)
df_incidents['_SERVICE_OFFERING'] = df_incidents['SERVICE_OFFERING'].pipe(clean_service_offering_column)

## 🔹 Filtro de grupos no válidos (A.TEC.ES)

- **Función `filtrar_grupos(df, columnas, fuente, casos_audit, casos_audit_descrip, clave_base)`**  
  - Revisa si los valores de las columnas especificadas **no comienzan por** `"A.TEC.ES."`.  
  - Filtra esas filas y añade metadatos:
    - `_FUENTE`: nombre de la tabla origen.  
    - `_TEST`: columna evaluada + condición aplicada.  

- **Aplicaciones en datasets PBI**  
  - `df_created_incidents`: verifica `"CREATOR_GROUP"` y `"ASSIGNMENT_GROUP"`.  
  - `df_resolved_incidents`: verifica `"CREATOR_GROUP"` y `"ASSIGNMENT_GROUP"`.  


In [ ]:
# def filtrar_grupos(df, columnas, fuente, casos_audit, casos_audit_descrip, clave_base):
#     """
#     Filtra filas de df donde las columnas no empiezan por 'A.TEC.ES.'.
#     Añade metadatos y actualiza los diccionarios de auditoría.
    
#     Args:
#         df (pd.DataFrame): dataframe de entrada
#         columnas (list): columnas a verificar ['CREATOR_GROUP','ASSIGNMENT_GROUP',...]
#         fuente (str): nombre de la fuente para asignar en la columna '_FUENTE'
#         casos_audit (dict): diccionario donde se guarda el resultado
#         casos_audit_descrip (dict): diccionario con descripciones
#         clave_base (str): clave base para usar en el dict (ej: 'PBI-INC_CreaNo_A-TEC-ES')
#     """
#     resultados = []
#     for col in columnas:
#         mask = ~df[col].str.startswith("A.TEC.ES.", na=False)
#         resultados.append(
#             df[mask].assign(_FUENTE=fuente, _TEST=f"{col} not start with A.TEC.ES.")
#         )

#     result = pd.concat(resultados, axis=0)
#     casos_audit[clave_base] = result
#     casos_audit_descrip[clave_base] = f"{clave_base}: Filtrado por {', '.join(columnas)}"
#     return casos_audit, casos_audit_descrip


# # df_created_incidents
# casos_audit, casos_audit_descrip = filtrar_grupos(
#     df_created_incidents,
#     ["CREATOR_GROUP", "ASSIGNMENT_GROUP"],
#     fuente="df_created_incidentsPBI",
#     casos_audit=casos_audit,
#     casos_audit_descrip=casos_audit_descrip,
#     clave_base="PBI-INC_CreaNo_A-TEC-ES",
# )

# # df_resolved_incidents
# casos_audit, casos_audit_descrip = filtrar_grupos(
#     df_resolved_incidents,
#     ["CREATOR_GROUP", "ASSIGNMENT_GROUP"],
#     fuente="df_resolved_incidentsPBI",
#     casos_audit=casos_audit,
#     casos_audit_descrip=casos_audit_descrip,
#     clave_base="PBI-INC_ResNo_A-TEC-ES",
# )

In [ ]:
def comparar_cause_code(df_created, df_resolved):
    """
    Compara los CAUSE_CODE entre incidentes creados y resueltos.

    Args:
        df_created (pd.DataFrame): DataFrame de incidentes creados
        df_resolved (pd.DataFrame): DataFrame de incidentes resueltos
        casos_audit (dict): Diccionario donde se guarda el resultado

    Returns:
        dict: casos_audit actualizado
    """

    def preparar(df, fuente):
        return (
            df.CAUSE_CODE.value_counts()
            .reset_index()
            .assign(
                _PCT=lambda x: x["count"] / x["count"].sum(),
                _FUENTE=fuente,
            )
        )

    df_created_proc = preparar(df_created, "df_created_incidentsPBI")
    df_resolved_proc = preparar(df_resolved, "df_resolved_incidentsPBI")

    merged_df = df_created_proc.merge(
        df_resolved_proc,
        on="CAUSE_CODE",
        how="outer",
        suffixes=("_created", "_resolved"),
    )

    result = (
        merged_df.sort_values(by="count_resolved", ascending=False)
        .reset_index(drop=True)
    )

    result["created_minus_resolved"] = (
        result["count_created"].fillna(0) - result["count_resolved"].fillna(0)
    )
    
    return result

# df_created_incidents vs df_resolved_incidents
key="PBI-INC_CreaVsRes"
casos_audit[key] = comparar_cause_code(df_created_incidents, df_resolved_incidents)
casos_audit_descrip[key] = "Comparación de CAUSE_CODE entre incidentes creados y resueltos (PowerBI)"

In [ ]:
# def filtrar_cause_code(df_created, df_resolved):
#     """
#     Filtra registros donde CAUSE_CODE NO empieza por 'Works as designed'
#     en incidentes creados y resueltos.

#     Args:
#         df_created (pd.DataFrame): DataFrame de incidentes creados
#         df_resolved (pd.DataFrame): DataFrame de incidentes resueltos
#         casos_audit (dict): diccionario donde guardar el resultado
#         key (str): clave para guardar en casos_audit

#     Returns:
#         dict: casos_audit actualizado
#     """

#     def preparar(df, fuente, rename_map=None):
#         mask = ~df["CAUSE_CODE"].str.startswith("Works as designed", na=False)
#         df_out = df[mask].assign(
#             _FUENTE=fuente,
#             _TEST="CAUSE_CODE dont start with Works as designed"
#         )
#         if rename_map:
#             df_out = df_out.rename(columns=rename_map)
#         return df_out

#     result_created = preparar(
#         df_created, 
#         "df_created_incidentsPBI"
#     )
#     result_resolved = preparar(
#         df_resolved,
#         "df_resolved_incidentsPBI"
#     )

#     result = pd.concat([result_created, result_resolved], axis=0)
    
#     return result

# # df_created_incidents vs df_resolved_incidents
# key="PBI-INC_CauseNo_WorkAsDesigned"
# casos_audit[key] = filtrar_cause_code(df_created_incidents, df_resolved_incidents)
# casos_audit_descrip[key] = "Comparación de CAUSE_CODE entre incidentes creados y resueltos (PowerBI)"

In [ ]:
# print(casos_audit.keys())
# print(casos_audit_descrip.keys())
# key="PBI-INC_CauseNo_WorkAsDesigned"

# display(casos_audit[key].head(),casos_audit[key].tail())

## 🔹 Filtro de incidentes de grupos españoles

- **Función `filtrar_incidentes_es(df_created, df_resolved, df_incidents, grupos_cols)`**  
  - Identifica incidentes relacionados con **grupos españoles**, usando información de PowerBI.  
  - Pasos principales:
    1. **Recolecta grupos únicos** de interés desde `df_created_incidents` y `df_resolved_incidents`.  
    2. **Filtra `df_incidents`** si alguno coincide en:
       - `ASSIGNMENT_GROUP`  
       - `LAST_ASSIGNMENT_GROUP`  
       - `CREATOR_GROUP`  
    3. Elimina incidentes sin `NUMBER`.  
    4. Ordena resultados con `sort_values_of_incidents`.  

- **Retorno**  
  - `df_filtrado`: incidentes válidos pertenecientes a grupos españoles.  
  - `es_groups`: lista de grupos únicos identificados.  

In [ ]:
# def filtrar_incidentes_es(df_created, df_resolved, df_incidents, grupos_cols=None):
#     """
#     Filtra incidentes relacionados con grupos españoles (según PowerBI).

#     Args:
#         df_created (pd.DataFrame): incidentes creados
#         df_resolved (pd.DataFrame): incidentes resueltos
#         df_incidents (pd.DataFrame): dataset completo de incidentes
#         sort_func (callable): función para ordenar los incidentes
#         grupos_cols (list): columnas donde buscar grupos (default=['ASSIGNMENT_GROUP','CREATOR_GROUP'])

#     Returns:
#         pd.DataFrame: df_incidents filtrado y ordenado
#     """

#     if grupos_cols is None:
#         grupos_cols = ["ASSIGNMENT_GROUP", "CREATOR_GROUP"]

#     # 1. Obtener todos los grupos únicos de interés
#     es_groups = pd.concat([
#         df_created[grupos_cols],
#         df_resolved[grupos_cols]
#     ], axis=0).stack().dropna().unique().tolist()

#     # 2. Filtrar incidentes cuyos grupos coinciden
#     mask = (
#         df_incidents["ASSIGNMENT_GROUP"].isin(es_groups)
#         | df_incidents["LAST_ASSIGNMENT_GROUP"].isin(es_groups)
#         | df_incidents["CREATOR_GROUP"].isin(es_groups)
#     )
#     df_lost = df_incidents.loc[~mask]
#     df_filtrado = df_incidents.loc[mask]
#     df_filtrado = df_filtrado[df_filtrado["NUMBER"].notna()]  # asegurar claves válidas
#     df_filtrado = df_filtrado.pipe(sort_values_of_incidents)

#     return df_filtrado, df_lost, es_groups


# df_incidents, df_lost, es_groups = filtrar_incidentes_es(df_created_incidents, df_resolved_incidents, df_incidents)
# try:
#     del df_lost
# except NameError:
#     pass

In [ ]:
# trace_changes['4_INC_Filtro_GruposRefEnPBI'] = print_shapes()

In [ ]:
def load_or_process_incidents(intermediate_data, df_incidents, load_from_pickle=False):
    """
    Carga incidentes desde pickle o los procesa y guarda.

    Args:
        intermediate_data (str): carpeta donde guardar/cargar pickles
        df_incidents (pd.DataFrame): dataset base de incidentes (si no hay pickle)
        load_from_pickle (bool): si True intenta cargar desde pickle
    """
    path = os.path.join(intermediate_data, "incidents.pickle")
    path_created = os.path.join(intermediate_data, "created_incidents.pickle")

    if load_from_pickle and os.path.exists(path):
        df_incidents = pd.read_pickle(path)

    else:
        # Procesamiento encadenado con .pipe para mayor claridad
        df_incidents = (
            df_incidents.pipe(get_inc_count_rows)
            .pipe(get_COLvals_countUnique_per_inc, func_col="PROBLEM")
            .pipe(get_COLvals_list_per_inc, func_col="PROBLEM")
            .pipe(get_COLvals_countUnique_per_inc, func_col="LAST_ASSIGNMENT_GROUP")
            .pipe(get_COLvals_list_per_inc, func_col="LAST_ASSIGNMENT_GROUP")
            .pipe(get_COLvals_countUnique_per_inc, func_col="ASSIGNMENT_GROUP")
            .pipe(get_bool_comparation, "LAST_ASSIGNMENT_GROUP", "ASSIGNMENT_GROUP")
            .pipe(get_bool_comparation, "CURRENT_PRIORITY", "PRIORITY")
            .pipe(get_bool_comparation, "HIGHEST_PRIORITY", "CURRENT_PRIORITY")
            .pipe(flag_state_change)
            .pipe(clean_LAST_ASSIGNMENT_GROUP)
            .pipe(clean_ASSIGNMENT_GROUP)
        )

        # Guardar
        df_incidents.to_pickle(path)


    return df_incidents


df_incidents = load_or_process_incidents(intermediate_data, df_incidents, False)

In [ ]:
df_created_incidents

In [ ]:
trace_changes['5_INC_FlagStateChange'] = print_shapes()

In [ ]:
log_separator(title="Section: [MAIN GET GROUPS] Description: [Get groups based on INCIDENT-PROBLEM]")
log_separator("TABLE: 'df_incidents'", level=3)

df_incidents = set_pct_completed(df_incidents)
df_incidents = process_incidents(df_incidents)

# Filtrar directamente donde _PROBLEM_TO_NULL == True
key = "INC_ProblemToNull"
casos_audit[key] = df_incidents.query("_PROBLEM_TO_NULL == True")
casos_audit_descrip[key] = "Incident que en UPDATED pasa de tener PROBLEM a NULL"

In [ ]:
trace_changes['6_INC_FromProblemToNull'] = print_shapes()

In [ ]:
def analizar_ticket_relations(dfs, relations):
    """
    Analiza relaciones entre tickets en varios DataFrames (parent-child).

    Args:
        dfs (dict): diccionario {nombre: DataFrame}
        relations (dict): diccionario {nombre: lista de pares de columnas}
        log_separator (callable): función para loggear
        logger (logging.Logger): logger activo
    """
    log_separator(
        title="Section: [MAIN TEST TICKET RELATION] Description: [DataFrame ticket relation analysis (parent-child)]"
    )
    logger.info(f"The relations are {relations}")

    result = {}
    for name, df in dfs.items():
        log_separator(f"TABLE: '{name}'", 3)
        pairs = relations.get(name, [])
        result[name] = log_and_get_nuniques(df, name, pairs)

    return result


dfs = {
    "df_incidents": df_incidents,
    "df_problem": df_problem,
    "df_problem_tasks": df_problem_tasks,
}

relations = {
    "df_incidents": [("NUMBER", "PROBLEM"), ("PROBLEM", "NUMBER")],
    "df_problem": [("NUMBER", "CREATED_OUT_OF_INCIDENT"), ("CREATED_OUT_OF_INCIDENT", "NUMBER")],
    "df_problem_tasks": [("NUMBER", "PROBLEM"), ("PROBLEM", "NUMBER")],
}

processed_dfs = analizar_ticket_relations(dfs, relations)

# ahora puedes hacer:
df_incidents = processed_dfs["df_incidents"]
df_problem = processed_dfs["df_problem"]
df_problem_tasks = processed_dfs["df_problem_tasks"]

try:
    del dfs, processed_dfs, relations
except Exception as e:
    logger.error(f"Error al eliminar variables: {e}")

In [ ]:
trace_changes['7_ALL_GetRelations'] = print_shapes()

In [ ]:
# result make a mask group by number and get the NUMBERS where any of _CHANGE_IN_ROW its rows has True
df_incidents = flag_PARENT_change(df_incidents)
mask = df_incidents[df_incidents['_CHANGE_IN_ROW'] == True].NUMBER.unique()
result = df_incidents[df_incidents.NUMBER.isin(mask)]
cols = [
    'NUMBER',
    'UPDATED',
    'PARENT_INCIDENT',
    'CHILD_INCIDENTS',
    '_CHANGE_IN_ROW',
    '_TYPE_CHANGE',
    '_SET_TICKET_PARENTS',
    '_TOTAL_CHANGES',
]

casos_audit['INC_ParentChanges'] = result.pipe(set_df_column_order, cols).pipe(
    sort_values_of_incidents, '_TOTAL_CHANGES'
)

casos_audit['FLAG_STATE_CHANGE'] = df_incidents[
    df_incidents['_IMPROPER_STATE_COUNTS'] > 0
]

inc_with_childs = df_incidents[df_incidents['CHILD_INCIDENTS'] != 0].NUMBER.unique()
casos_audit['INC_with_CHILD'] = df_incidents[
    df_incidents['NUMBER'].isin(inc_with_childs)
]

casos_audit['INC_CompanyChanges'] = (
    df_incidents.groupby('NUMBER')['COMPANY']
    .agg([pd.Series.nunique, set])
    .query('nunique > 1')
    .reset_index()
)

df_assignment_groups = load_confluence()

df_incidents = df_incidents.merge(
    df_assignment_groups,
    on='LAST_ASSIGNMENT_GROUP',
    how='left',
    suffixes=('', '_CONFLUENCE'),
    indicator='_mergeAG',
)
df_created_incidents = df_created_incidents.merge(
    (
        df_assignment_groups.rename(
            columns={'LAST_ASSIGNMENT_GROUP': 'ASSIGNMENT_GROUP'}
        )
    ),
    on='ASSIGNMENT_GROUP',
    how='left',
    suffixes=('', '_CONFLUENCE'),
    indicator='_mergeAG',
)

casos_audit['ASSIG_GROUP_NO_CONFLUENCE'] = df_incidents.query(
    '_mergeAG == "left_only"'
).drop_duplicates(keep='first')

casos_audit['ASSIG_GROUP_PBI_NO_CONFLUENCE'] = df_created_incidents.query(
    '_mergeAG == "left_only"'
).drop_duplicates(keep='first')

try:
    del df_assignment_groups
except:
    pass

In [ ]:
trace_changes['8_Confluence'] = print_shapes()

In [ ]:
mask = (
    df_incidents['REASSIGNMENT_COUNT']
    != df_incidents['_ASSIGNMENT_GROUP_COUNT_UNIQUE_PER_INC']
)
casos_audit['INC_assigment_flags'] = (
    df_incidents[mask][
        [
            'NUMBER',
            'UPDATED',
            'LAST_ASSIGNMENT_GROUP',
            'ASSIGNMENT_GROUP',
            '_LAST_ASSIGNMENT_GROUP_COUNT_UNIQUE_PER_INC',
            '_ASSIGNMENT_GROUP_COUNT_UNIQUE_PER_INC',
            'REASSIGNMENT_COUNT',
            '_INC_COUNT_ROWS',
        ]
    ]
    .pipe(sort_values_of_incidents)
    .loc[
        lambda x: x['REASSIGNMENT_COUNT']
        != x['_ASSIGNMENT_GROUP_COUNT_UNIQUE_PER_INC'].map({0: 0, 1: 0})
    ]
)

In [ ]:

log_separator(
    title="Section: [TRANSFORM INCIDENTS DF UNIQUE NUMBER (COMPILE UPDATES)] Description: [Data incident transformation from multiple rows of tickets with updates to a single row]")
load_from_pickle = False
try:
    log_separator("TABLE: 'df_incidents'", 3)
    pickle_path = os.path.join(intermediate_data, "Data_INC_uniques.pkl")
    if os.path.exists(pickle_path) and load_from_pickle:
        logger.info("Loading Data_INC_uniques from pickle")
        df_incidents_grouped = pd.read_pickle(pickle_path)
    else:
        df_incidents_grouped = make_uniques(
            df_incidents, principal_col='NUMBER', link_to_column='PROBLEM')
        logger.info("Saving Data_INC_uniques to pickle")
        df_incidents_grouped.to_pickle(pickle_path)
    
    df_incidents_grouped = df_incidents_grouped.pipe(create_dt_time_wvariants)
    

except Exception as e:
    function_name = "MAIN TRANSFORM INCIDENTS DF UNIQUE NUMBER (COMPILE UPDATES)"
    message = f"[ERROR]: {function_name} {e}"
    print(message)
    logger.error(message)
    raise e

In [ ]:
# # filtrar hasta despues de haber cerado df_incidents_grouped
# # exclude NUMBER where CAUSE_CODE == 'Works as designed ' 
# df_incidents_grouped = df_incidents_grouped[~df_incidents_grouped['CAUSE_CODE'].str.contains('Works as designed', na=False)]

# # exclude NUMBER where valor '.BUS.'
# # en campos ['LAST_ASSIGNMENT_GROUP','ASSIGNMENT_GROUP','CREATOR_GROUP']
# for col in ['LAST_ASSIGNMENT_GROUP','ASSIGNMENT_GROUP','CREATOR_GROUP']:
#     df_incidents_grouped = df_incidents_grouped[~df_incidents_grouped[col].str.contains('.BUS.', na=False)]

# # exclude
# exclude = ['ALLIANZ TECHNOLOGY',
#  'ALLIANZ TECHNOLOGY EXTERNAL',
#  'ALLIANZ TECHNOLOGY IBEROLATAM BRANCH',
#  'ALLIANZ TECHNOLOGY IBEROLATAM EXTERNAL',
#  'ALLIANZ TECHNOLOGY GDN INTERNAL']
# df_incidents_grouped = df_incidents_grouped[~df_incidents_grouped['COMPANY'].isin(exclude)]

In [ ]:
trace_changes['9_Make_Unique'] = print_shapes()

In [ ]:
# Define masks based on conditions
mask_1 = df_incidents_grouped['PARENT_INCIDENT'] != ''
mask_2 = df_incidents_grouped['FCR'] == True

# strings = ['GLOB', 'SERVICEDESK', '_SD_', '.SD.', 'WORKPL', 'WPS', 'WORK_PL', 'WORK.PL']
# mask_3 = (
#     df_incidents_grouped['LAST_ASSIGNMENT_GROUP']
#     .str.upper()
#     .str.contains('|'.join(strings), na=False)
# )

# Create new columns based on the masks
df_incidents_grouped.loc[mask_1, '_NOT_CHILD'] = True
df_incidents_grouped.loc[~mask_1, '_NOT_CHILD'] = False

df_incidents_grouped.loc[mask_2, '_NOT_FCR'] = True
df_incidents_grouped.loc[~mask_2, '_NOT_FCR'] = False

# df_incidents_grouped['_ASSIGNMENT_Glob_SerDes_WorPla'] = False
# df_incidents_grouped.loc[mask_3, '_ASSIGNMENT_Glob_SerDes_WorPla'] = True

df_incidents_grouped['_FILTER_OUT'] = df_incidents_grouped[
    [
        '_NOT_CHILD', 
        '_NOT_FCR', 
        # '_ASSIGNMENT_Glob_SerDes_WorPla'
    ]
].any(axis=1)

df_incidents_grouped_lost = df_incidents_grouped[df_incidents_grouped['_FILTER_OUT']]
df_incidents_grouped = df_incidents_grouped[~df_incidents_grouped['_FILTER_OUT']]

mask = df_problem['CREATED_OUT_OF_INCIDENT'].isin(
    df_incidents_grouped['NUMBER']
) | df_problem['NUMBER'].isin(df_incidents_grouped['PROBLEM'])
df_problem_lost = df_problem[~mask]
df_problem = df_problem[mask]

mask = df_problem_tasks['PROBLEM'].isin(
    df_incidents_grouped['PROBLEM']
) | df_problem_tasks['PROBLEM'].isin(df_problem['NUMBER'])
df_problem_tasks_lost = df_problem_tasks[~mask]
df_problem_tasks = df_problem_tasks[mask]

try:
    del df_problem_tasks_lost, df_problem_lost, df_incidents_grouped_lost
except NameError:
    pass

In [ ]:
trace_changes['10_FiltroParentFCR'] = print_shapes()

In [ ]:
# allianz_blue = '#0055A4'
# allianz_light_blue = '#A3C7E4'
# allianz_gray = '#6C757D'
# for col in [col for col in df_incidents_grouped.select_dtypes(exclude=['number','datetime']).columns if df_incidents_grouped[col].nunique() <= 10]:
#     plt.figure(figsize=(10, 5))  # Set figure size for better visibility
#     df_incidents_grouped[col].value_counts(normalize=True, dropna=False).pipe(lambda ser: ser[ser>0]).plot.barh(
#         color=allianz_blue, edgecolor=allianz_light_blue
#     )
#     plt.title(f'Distribution of {col}', fontsize=14, fontweight='bold', color=allianz_gray)
#     plt.xlabel('Frequency', fontsize=12, color=allianz_gray)
#     plt.ylabel('Values', fontsize=12, color=allianz_gray)
#     plt.grid(axis='x', linestyle='--', linewidth=0.7, color=allianz_light_blue)
#     plt.tight_layout()
#     plt.show()  # Show plot for each column

In [ ]:
# for col in [col for col in df_incidents_grouped.select_dtypes(exclude=['number', 'datetime']).columns 
#             if df_incidents_grouped[col].nunique() >= 10 and df_incidents_grouped[col].nunique() <= 30]:
#     plt.figure(figsize=(10, 5))  # Set figure size for better visibility
#     df_incidents_grouped[col].value_counts(normalize=True, dropna=False).pipe(lambda ser: ser[ser>0]).plot.bar(
#         color=allianz_blue, edgecolor=allianz_light_blue
#     )
#     plt.title(f'Distribution of {col}', fontsize=14, fontweight='bold', color=allianz_gray)
#     plt.xlabel('Frequency', fontsize=12, color=allianz_gray)
#     plt.ylabel('Values', fontsize=12, color=allianz_gray)
#     plt.grid(axis='x', linestyle='--', linewidth=0.7, color=allianz_light_blue)
#     plt.tight_layout()
#     plt.show()  # Show plot for each column

In [ ]:
# # kde o density
# for col in [col for col in df_incidents_grouped.select_dtypes(include=['number']).columns]:
#     plt.figure(figsize=(10, 5))  # Set figure size for better visibility
#     df_incidents_grouped[col].plot.hist(
#         bins=25,
#         color=allianz_blue, 
#         #edgecolor=allianz_light_blue
#     )
#     plt.title(f'Distribution of {col}', fontsize=14, fontweight='bold', color=allianz_gray)
#     plt.xlabel('Frequency', fontsize=12, color=allianz_gray)
#     plt.ylabel('Values', fontsize=12, color=allianz_gray)
#     plt.grid(axis='x', linestyle='--', linewidth=0.7, color=allianz_light_blue)
#     plt.tight_layout()
#     plt.show()  # Show plot for each column

In [ ]:
load_from_pickle = False
pickle_path = os.path.join(intermediate_data, "df_incidents_grouped_pnc.pkl")
if os.path.exists(pickle_path) and load_from_pickle:
    logger.info("Loading df_incidents_grouped from pickle")
    df_incidents_grouped = pd.read_pickle(pickle_path)
    df_problem = pd.read_pickle(os.path.join(intermediate_data, "df_problem_pnc.pkl"))
    df_problem_tasks = pd.read_pickle(
        os.path.join(intermediate_data, "df_problem_tasks_pnc.pkl")
    )
else:
    logger.info("Saving df_incidents_grouped to pickle")
    # INCIDENTS
    date_pairs = time_cols_pairs['df_incidents']
    df_incidents_grouped = process_not_closed_business_vectorized(
        df_incidents_grouped, date_pairs
    )
    df_incidents_grouped.to_pickle(pickle_path)

    # PROBLEMS
    date_pairs = time_cols_pairs['df_problem']
    df_problem = process_not_closed_business_vectorized(df_problem, date_pairs)
    df_problem.to_pickle(os.path.join(intermediate_data, "df_problem_pnc.pkl"))

    # PROBLEM TASKS
    date_pairs = time_cols_pairs['df_problem_tasks']
    df_problem_tasks = process_not_closed_business_vectorized(
        df_problem_tasks, date_pairs
    )
    df_problem_tasks.to_pickle(
        os.path.join(intermediate_data, "df_problem_tasks_pnc.pkl")
    )

In [ ]:
trace_changes['11_NotClosed'] = print_shapes()

In [ ]:
df_incidents_grouped['COMPANY'] = df_incidents_grouped['COMPANY'].apply(
    standardize_company_name
)
df_incidents_grouped['_CLOSE_NOTES'] = clean_close_notes(
    df_incidents_grouped['CLOSE_NOTES']
)
df_incidents_grouped['_nullProblem'] = df_incidents_grouped['PROBLEM'].isna()

df_incidents_grouped = exceed_bands_priority_CRE_RES_optimized(
    df_incidents_grouped, 'CURRENT_PRIORITY'
)  # PRIORITY | HIGHEST_PRIORITY

# PROBLEMS
df_problem = exceed_bands_priority_CRE_RES_optimized(df_problem, 'CURRENT_PRIORITY')

In [ ]:
cp2 = ['INC29879295','INC31431639','INC31422945']
casos_audit['2025_CP"'] = df_incidents[df_incidents['NUMBER'].isin(cp2)]

In [ ]:
trace_changes['12_Bands_Exceed'] = print_shapes()

In [ ]:
df_incidents_grouped

In [ ]:
casos_audit['Priority_ProblemNULL_Year'] = df_incidents_grouped.pivot_table(
    index='CURRENT_PRIORITY',
    columns='_CREATED_year',
    values='_nullProblem',
    aggfunc='sum',
    fill_value=0,
    margins=True,
    margins_name='Total',
).reset_index()

# prueba:
# (df_incidents_grouped[(df_incidents_grouped['CURRENT_PRIORITY']=='P3') &
#                      (df_incidents_grouped['PROBLEM'].isna()) &
#                      (df_incidents_grouped['_CREATED_year']==2025)
#                      ]
# ).reset_index(drop=True)

In [ ]:
# Convert to days
# df_incidents_grouped['_DIFB_CREATED_RESOLVED_days'] = df_incidents_grouped['_DIFB_CREATED_RESOLVED'].dt.total_seconds() / (24 * 60 * 60)

# Or convert to hours
# df_incidents_grouped['_DIFB_CREATED_RESOLVED_hours'] = df_incidents_grouped['_DIFB_CREATED_RESOLVED'].dt.total_seconds() / 3600

# Or convert to seconds
# df_incidents_grouped['_DIFB_CREATED_RESOLVED_seconds'] = df_incidents_grouped['_DIFB_CREATED_RESOLVED'].dt.total_seconds()

# Create pivot table with different aggregations for different columns
casos_audit['PT_SLA_GROUPS'] = df_incidents_grouped.pivot_table(
    index=[
        'CURRENT_PRIORITY',
        'STATE',
        'LAST_ASSIGNMENT_GROUP',
        '_SLA_CURRENT_PRIORITY',
        '_SLA_THRESHOLD',
    ],
    columns='_CREATED_year',
    values=['_DIFB_CREATED_RESOLVED', 'NUMBER'],
    aggfunc={
        '_DIFB_CREATED_RESOLVED': ['mean', 'std', 'min', 'max', 'count'],
        'NUMBER': [lambda x: list(x), 'nunique'],
    },
    fill_value=0,
).reset_index()

#  margins=True,
#  margins_name='Total'

In [ ]:
# Campos con valores nulos
# casos_audit["VALORES_NULOS"] = df_incidents_grouped.isna().sum().reset_index(name='SUM').query("SUM > 0").assign(_PCT=lambda x: x['SUM'] / df_incidents_grouped.shape[0] * 100)
# Se han observado 3 casos AFFECTED_OES en blanco y compañia == 'ALLIANZ COMPANIA DE SEGUROS Y REASEGUROS SA'
casos_audit["AFFECTED_OES_nulos"] = df_incidents_grouped[
    df_incidents_grouped.filter(regex='AFF')['AFFECTED_OES'].isna()
]
# 4 incidententes con 'CAUSE_CODE' en blanco y 'STATE' = 'Closed'
mask = (df_incidents_grouped['CAUSE_CODE'].isna()) & (
    df_incidents_grouped['STATE'] == 'Closed'
)
casos_audit["CAUSE_CODE_nulos_Closed"] = df_incidents_grouped[mask][
    ['NUMBER', 'CLOSED', 'CAUSE_CODE', 'STATE']
]
# Campos con valores nulos en 'CAUSE_CODE'
mask = df_incidents_grouped['CAUSE_CODE'].isna()
casos_audit["CAUSE_CODE_nulos_Campos"] = append_sample_to_value_counts(
    df_incidents_grouped[mask].STATE, df_incidents_grouped, mask
).reset_index()
# 4 casos 'CAUSE_CODE'=='Other' y 'STATE'=='Closed'
mask = (~df_incidents_grouped['CAUSE_CODE'].isna()) & (
    df_incidents_grouped['STATE'] == 'Closed'
)
casos_audit["CAUSE_CODE_Other_Closed"] = (
    df_incidents_grouped[mask][['CAUSE_CODE', 'STATE']].value_counts().reset_index()
)
# casos audit_fail 'CLOSE_NOTES'== y 'STATE'=='Closed'
mask = (
    (~df_incidents_grouped['_CLOSE_NOTES'].isna())
    & (df_incidents_grouped['STATE'] == 'Closed')
    & (df_incidents_grouped['_CLOSE_NOTES'].str.len() < 15)
)
casos_audit["CLOSE_NOTES_fails_Closed"] = (
    df_incidents_grouped[mask][['_CLOSE_NOTES', 'STATE']].value_counts().reset_index()
)
# Análisis de correlación
casos_audit["corr_ASSIGMENT_SERVICE"] = (
    df_incidents_grouped[['LAST_ASSIGNMENT_GROUP', 'SERVICE_OFFERING']]
    .value_counts(dropna=False)
    .reset_index()
)
casos_audit["corr_SYMPTOM_SERVICE"] = (
    df_incidents_grouped[['SYMPTOM', 'SERVICE_OFFERING']]
    .value_counts(dropna=False)
    .reset_index()
)
casos_audit["corr_ASSIGMENT_SERVICE2"] = analyze_service_offerings(
    df_incidents_grouped, 'LAST_ASSIGNMENT_GROUP', 'SERVICE_OFFERING'
)
casos_audit["corr_SYMPTOM_SERVICE2"] = analyze_service_offerings(
    df_incidents_grouped, 'SYMPTOM', 'SERVICE_OFFERING'
)
casos_audit["corr_SYMPTOM_ASSIGMENT"] = analyze_service_offerings(
    df_incidents_grouped, 'SYMPTOM', 'LAST_ASSIGNMENT_GROUP'
)
# Aging
# casos_audit["AGING"] = df_incidents_grouped.groupby('STATE')[df_incidents_grouped.filter(like='_DIFB').columns].agg(['mean', 'std', 'count']).reset_index()
casos_audit["AGING_Business"] = (
    df_incidents_grouped.groupby(['STATE', 'CURRENT_PRIORITY'])[
        df_incidents_grouped.filter(like='_DIFB').columns
    ]
    .agg(['mean', 'std', 'count', 'min', 'max'])
    .reset_index()
)
# Incident tickets PRIORITY vs OUTAGE_BEGIN to CLOSED
# casos_audit["PRIORITY_OUTAGE_BEGIN_CLOSED"] = (
#     df_incidents_grouped.loc[df_incidents_grouped['STATE'] == 'Closed']
#     .groupby(['CURRENT_PRIORITY', 'STATE'])['_DIFB_CREATED_RESOLVED']
#     .agg(['mean', 'std', 'count', 'min', 'max'])
#     .reset_index()
# )

# Problems root_couse
dfs = []
for col in [
    'CURRENT_SYMPTOM',
    'SUBCLASSIFICATION',
    'CLASSIFICATION',
    'ROOT_CAUSE_KNOWN',
    'CLOSE_CODE',
]:
    value_counts = (
        df_problem.groupby('STATE')[col]
        .value_counts(dropna=False)
        .reset_index(name='count')
    )
    value_counts['_PCT'] = (
        value_counts['count']
        / value_counts.groupby('STATE')['count'].transform('sum')
        * 100
    )
    value_counts['_COL'] = col
    value_counts.columns = ['STATE', '_VALUE', '_COUNT', '_PCT', '_COL_GROUP']
    dfs.append(value_counts)
df_problems_root_cause = pd.concat(dfs, ignore_index=True)
del dfs

casos_audit["PROBLEMS_ROOT_CAUSE"] = df_problems_root_cause
# casos_audit["INCIDENTS_LOST"] = df_incidents_grouped_lost
# casos_audit["PROBLEMS_LOST"] = df_problem_lost
# casos_audit["PROBLEMS_TASK_LOST"] = df_problem_tasks_lost

In [ ]:
df_incidents_grouped[['LAST_ASSIGNMENT_GROUP', 'SERVICE_OFFERING','_SERVICE_OFFERING_OE']]

In [ ]:
### OUTAGE_BEGIN in STAGES Closed|Resolved
mask_1 = df_incidents_grouped['OUTAGE_BEGIN'].isna() & (df_incidents_grouped['STATE'] == 'Closed')
mask_2 = df_incidents_grouped['OUTAGE_BEGIN'].isna() & (df_incidents_grouped['STATE'] == 'Resolved')
combined = mask_1 | mask_2
# df_incidents_grouped[combined]
df_incidents_grouped['_OutBegNull_StateCloRes'] = combined

In [ ]:
missing_values_count = df_incidents_grouped.isna().sum()
missing_values_pct = df_incidents_grouped.isna().mean() * 100
missing_values_summary = pd.DataFrame({
    'count': missing_values_count,
    'percentage': missing_values_pct
})
columns_with_missing_values = missing_values_summary[missing_values_summary['count'] > 0]
sorted_columns = columns_with_missing_values.sort_values(by='count')
casos_audit['VALORES_NULOS_INC'] = sorted_columns.reset_index()


In [ ]:
missing_values_count = df_problem.isna().sum()
missing_values_pct = df_problem.isna().mean() * 100
missing_values_summary = pd.DataFrame({
    'count': missing_values_count,
    'percentage': missing_values_pct
})
columns_with_missing_values = missing_values_summary[missing_values_summary['count'] > 0]
sorted_columns = columns_with_missing_values.sort_values(by='count')
casos_audit['VALORES_NULOS_PRB'] = sorted_columns.reset_index()

In [ ]:
df_incidents_grouped.shape

In [ ]:
log_separator(title="Section: [TEST POWERBI DFS VS SERVICE NOW DFS] Description: [Add PowerBI ticket numbers comparison]")
try:
    # PowerBI dfs VS ServiceNow dfs
    # df_incidents_grouped = df_incidents_grouped.merge(df_created_incidents[['NUMBER']], on=['NUMBER'], how='left', suffixes=('', '_CRE_PBI'), indicator='_merge_CRE_PBI')
    # df_incidents_grouped = df_incidents_grouped.merge(df_resolved_incidents[['NUMBER']], on=['NUMBER'], how='left', suffixes=('', '_RES_PBI'), indicator='_merge_RES_PBI')
    # df_problem = df_problem.merge(df_problems_pbi[['NUMBER']], on=['NUMBER'], how='left', suffixes=('', '_PRB_PBI'), indicator='_merge_PRB_PBI')
    # # ------------------------------------------------------------
    # Extract unique incident numbers from different DataFrames
    powerbi_cre_inc_num = df_created_incidents['NUMBER'].dropna().unique()
    powerbi_cre_prb_num = df_created_incidents['PROBLEM'].dropna().unique()
    
    powerbi_res_inc_num = df_resolved_incidents['NUMBER'].dropna().unique()
    
    powerbi_prb_prb_num = df_problems_pbi['NUMBER'].dropna().unique()
    powerbi_prb_inc_num = df_problems_pbi['CREATED_OUT_OF_INCIDENT'].dropna().unique()
    # ------------------------------------------------------------
    # Create dictionaries for mapping
    powerbi_cre_inc_num = {num: num for num in powerbi_cre_inc_num}
    powerbi_cre_prb_num = {num: num for num in powerbi_cre_prb_num}
    
    powerbi_res_inc_num = {num: num for num in powerbi_res_inc_num}
    
    powerbi_prb_prb_num = {num: num for num in powerbi_prb_prb_num}
    powerbi_prb_inc_num = {num: num for num in powerbi_prb_inc_num}
    # ------------------------------------------------------------
    # Check if incident numbers are in the respective lists and log the results
    df_incidents_grouped["_CRE_INC_Match"] = df_incidents_grouped["NUMBER"].map(powerbi_cre_inc_num)
    df_incidents_grouped["_CRE_PRB_Match"] = df_incidents_grouped["PROBLEM"].map(powerbi_cre_prb_num)
    
    df_incidents_grouped["_RES_INC_Match"] = df_incidents_grouped["NUMBER"].map(powerbi_res_inc_num)
    
    df_incidents_grouped["_PRB_PRB_Match"] = df_incidents_grouped["PROBLEM"].map(powerbi_prb_prb_num)
    df_incidents_grouped["_PRB_INC_Match"] = df_incidents_grouped["NUMBER"].map(powerbi_prb_inc_num)
    # ------------------------------------------------------------
    df_problem["_PRB_PRB_Match"] = df_problem["NUMBER"].map(powerbi_prb_prb_num)
    df_problem["_PRB_INC_Match"] = df_problem["NUMBER"].map(powerbi_prb_inc_num)
    
    df_problem["_CRE_INC_Match"] = df_problem["CREATED_OUT_OF_INCIDENT"].map(powerbi_cre_inc_num)
    df_problem["_CRE_PRB_Match"] = df_problem["NUMBER"].map(powerbi_cre_prb_num)
    
    df_problem["_RES_INC_Match"] = df_problem["CREATED_OUT_OF_INCIDENT"].map(powerbi_res_inc_num)
    # ------------------------------------------------------------
    df_problem_tasks["_PRB_PRB_Match"] = df_problem_tasks["PROBLEM"].map(powerbi_prb_prb_num)
    df_problem_tasks["_CRE_PRB_Match"] = df_problem_tasks["PROBLEM"].map(powerbi_cre_prb_num)
    # ------------------------------------------------------------
    # Initialize result DataFrame
    df_result = pd.DataFrame()
    # Append matches to result DataFrame
    df_result = pd.concat([df_result, (df_incidents_grouped
                                       [['NUMBER', 'PROBLEM', '_CRE_INC_Match', '_CRE_PRB_Match', '_RES_INC_Match', '_PRB_PRB_Match', '_PRB_INC_Match']+df_incidents_grouped.filter(like="_PBI").columns.to_list()]
                                       .assign(Source='df_incidents_grouped')
                                       .rename(columns={'NUMBER': 'Incident Number', 
                                                        'PROBLEM': 'Problem Number'})
                                       )])
    # Append matches to result DataFrame
    df_result = pd.concat([df_result, (df_problem
                                       [['NUMBER', 'CREATED_OUT_OF_INCIDENT', '_CRE_INC_Match', '_CRE_PRB_Match', '_RES_INC_Match', '_PRB_PRB_Match', '_PRB_INC_Match']+df_problem.filter(like="_PBI").columns.to_list()]
                                       .assign(Source='df_problem')
                                       .rename(columns={'NUMBER':'Problem Number', 
                                                        'CREATED_OUT_OF_INCIDENT':'Incident Number'})
                                       )])
    # Append matches to result DataFrame
    df_result = pd.concat([df_result, (df_problem_tasks
                                       [['NUMBER', 'PROBLEM', '_PRB_PRB_Match', '_CRE_PRB_Match']]
                                       .assign(Source='df_problem_tasks')
                                       .rename(columns={'NUMBER':'Problem Task Number', 
                                                        'PROBLEM':'Problem Number'})
                                       )])
    cols = ['Source', 'Incident Number', 'Problem Number', 'Problem Task Number']
    df_result = df_result.pipe(set_df_column_order, start_cols=cols).pipe(clean_headers)
    display(df_result)
    casos_audit["vs_PBI"] = df_result

except Exception as e:
    function_name = "MAIN TEST POWERBI DFS VS SERVICE NOW DFS"
    message = f"[ERROR]: {function_name} {e}"
    print(message)
    logger.error(message)
    raise e

log_separator(title="Section: [MAIN INFO OF CHANGES] Description: [DataFrame changes (checkpoint)]", level=3)
try:
    transformed_columns = {
        "df_incidents": df_incidents.columns.tolist(),
        "df_problem": df_problem.columns.tolist(),
        "df_problem_tasks": df_problem_tasks.columns.tolist()
    }
    # ------------------------------------------------------------
    transformed_shapes = {
        "df_incidents": df_incidents.shape,
        "df_problem": df_problem.shape,
        "df_problem_tasks": df_problem_tasks.shape
    }
    # ------------------------------------------------------------
    message="On Section: [INFO OF CHANGES]"
    dict_transformed_cols = compare_cols_dicts(original_columns, transformed_columns, message=message)
    dict_transformed_shapes = compare_shapes_dicts(original_shapes, transformed_shapes, message=message)
    print(dict_transformed_cols)
    print(dict_transformed_shapes)
    
except Exception as e:
    function_name = "MAIN INFO OF CHANGES"
    message = f"[ERROR]: {function_name} {e}"
    print(message)
    logger.error(message)
    raise e

In [ ]:
# def filter_those_with_more_than_one_coma(series):
#     return series[series.str.count(',') > 1]
# df_incidents_grouped._CHANGES_IN_LINK.pipe(filter_those_with_more_than_one_coma)

In [ ]:
# ISS09
casos_audit['PTASK_TYPE_pt_year'] = (df_problem_tasks
                                     .pivot_table(
    index=['TYPE_OF_TASK'],
    columns='_CREATED_year',
    values='NUMBER',
    aggfunc='count',
    dropna=True
)
    .assign(TOTAL=lambda x: x[x.select_dtypes(include='number').columns].sum(axis=1))
    .sort_values(by='TOTAL', ascending=False)
    .assign(PCT=lambda x: 100 * np.round(x.TOTAL / x.TOTAL.sum(), 4))
)

In [ ]:
df_problem_tasks_Lessons = (
    df_problem_tasks[['NUMBER','CLOSE_NOTES','TYPE_OF_TASK','_CREATED_year','SHORT_DESCRIPTION']]
)

# tabulate_df(df_problem_tasks_Lessons[df_problem_tasks_Lessons['_CREATED_year'] == 2025])
display(df_problem_tasks_Lessons[df_problem_tasks_Lessons['_CREATED_year'] == 2025])

grouped_tasks_2025 = (df_problem_tasks_Lessons[df_problem_tasks_Lessons['_CREATED_year'] == 2025]
               .groupby('TYPE_OF_TASK').agg(
                   Total_Tasks=('NUMBER', 'count'),
                   Empty_CLOSE_NOTES=('CLOSE_NOTES', lambda x: x.isna().sum())
               ).sort_values(by='Total_Tasks', ascending=False)
)

# Calcular el porcentaje de tareas con CLOSE_NOTES vacías
grouped_tasks_2025['Percentage_Empty_CLOSE_NOTES'] = (
    grouped_tasks_2025['Empty_CLOSE_NOTES'] / grouped_tasks_2025['Total_Tasks'] * 100
)

# Calcular el porcentaje de cada TYPE_OF_TASK respecto al total de tareas en 2025
total_tasks_2025 = grouped_tasks_2025['Total_Tasks'].sum()
grouped_tasks_2025['Percentage_TYPE_OF_TASK'] = (
    grouped_tasks_2025['Total_Tasks'] / total_tasks_2025 * 100
)

# Formatear los porcentajes para mostrar como valores con dos decimales
grouped_tasks_2025['Percentage_Empty_CLOSE_NOTES'] = grouped_tasks_2025['Percentage_Empty_CLOSE_NOTES'].map('{:.2f}%'.format)
grouped_tasks_2025['Percentage_TYPE_OF_TASK'] = grouped_tasks_2025['Percentage_TYPE_OF_TASK'].map('{:.2f}%'.format)
grouped_tasks_2025.reset_index()

# tabulate_df(grouped_tasks_2025)
display(grouped_tasks_2025)

In [ ]:
plot_dates_periods(df_incidents_grouped)

In [ ]:
keywords = [
    'LESSON','LECCION', 'LECCIÓN',
    # 'ROOT CAUSE', 'CAUSA RAIZ'
]

df_incidents_grouped = df_incidents_grouped.assign(_word_Lesson= lambda x: x.apply(find_words_in_row, words=keywords, axis=1))

keywords = [
    # 'LESSON','LECCION', 'LECCIÓN',
    'ROOT CAUSE', 'CAUSA RAIZ'
]

df_incidents_grouped = df_incidents_grouped.assign(_word_RootCause= lambda x: x.apply(find_words_in_row, words=keywords, axis=1))

In [ ]:
# logger.info("process_dataframe: df_problem")
process_dataframe_stage(df_problem, ['CURRENT_PRIORITY'], examples=True)

# logger.info("process_dataframe: df_problem_tasks")
# process_dataframe(df_problem_tasks, ['TYPE_OF_TASK'], examples=True)

In [ ]:

# cols = ['ASSIGNMENT_GROUP', 'AFFECTED_OES', 'PRIORITY','STATE','CREATED','RESOLVED','CLOSE_NOTES','ROOT_CAUSE_KNOWN','PROBLEM_TASKS','CLASSIFICATION']
result = find_columns_value_match(df_problem)[['NUMBER','CREATED_OUT_OF_INCIDENT','_LINKED_TICKETS']]


result = get_LINKED_TICKETS_summary(result)
display(result)
casos_audit["LINKED_TICKETS_PBR"] = result

In [ ]:
# cols = ['ASSIGNMENT_GROUP', 'AFFECTED_OES', 'PRIORITY','STATE','CREATED','RESOLVED','CLOSE_NOTES','ROOT_CAUSE_KNOWN','PROBLEM_TASKS','CLASSIFICATION']
result = find_columns_value_match(df_incidents_grouped)[['NUMBER','PROBLEM','_LINKED_TICKETS']]

result = get_LINKED_TICKETS_summary(result)
display(result)
casos_audit["LINKED_TICKETS_INC"] = result

In [ ]:
df_incidents_grouped.shape

In [ ]:
df_incidents_grouped = find_columns_value_match(df_incidents_grouped, 
                         ignore_columns=['NUMBER', 'CREATED_OUT_OF_INCIDENT', 'PROBLEM', '_PROBLEM_LIST_PER_INC','_CHANGES_DICT', '_CRE_INC_Match', 
                                         '_CRE_PRB_Match', '_RES_INC_Match', '_PRB_PRB_Match', '_PRB_INC_Match','_values_NUMBER','_values_PROBLEM']
                         )

In [ ]:
df_incidents_grouped.shape

In [ ]:
# df_incidents_grouped.query('_LINKED_TICKETS_COUNT > 0')

In [ ]:
# ejemplos_problems = pd.read_excel(raw_data+'/EJEMPLOS_QA.xlsx').PROBLEM_NUMBER.to_list()
# print(ejemplos_problems)
# ejemplos_incidents = df_problem.loc[lambda x: x['NUMBER'].isin(ejemplos_problems)].CREATED_OUT_OF_INCIDENT.unique()
# print(ejemplos_incidents)

# # save filtered DataFrames to Excel
# with pd.ExcelWriter(processed_data + '/filtered_examples.xlsx') as writer:
#     df_problem.loc[lambda x: x['NUMBER'].isin(ejemplos_problems)].to_excel(writer, sheet_name='Filtered_Problems', index=False)
#     df_incidents.loc[lambda x: x['NUMBER'].isin(ejemplos_incidents)].to_excel(writer, sheet_name='Filtered_Incidents', index=False)
#     df_problem_tasks.loc[lambda x: x['PROBLEM'].isin(ejemplos_problems)].to_excel(writer, sheet_name='Filtered_Problem_Tasks', index=False)

In [ ]:

class IncidentAnalyzer:
    def __init__(self, df, config):
        self.df = df
        self.config = config

    def check_duplicates(self, column_name):
        logger.info(f"Config keys used: {column_name}, problem_column")
        if self.df[self.config[column_name]].duplicated().any():
            logger.warning(f"Duplicate {self.config[column_name]} found in {column_name}.")
            if self.config[column_name] == self.config['problem_column']:
                dup_df = (self.df[lambda x: x[self.config[column_name]].duplicated(keep=False)]
                          .groupby(self.config[column_name])[self.config['number_column']]
                            .agg(
                                nunique=lambda x: x.nunique(dropna=False),
                                values=lambda x: ','.join(set(val for val in x if pd.notnull(val))),
                                contains_null=lambda x: x.isnull().any()
                            )
                          .reset_index()
                          )
                logger.warning("\n" + tabulate(dup_df, headers='keys', tablefmt='psql'))
            if self.config[column_name] == self.config['number_column']:
                dup_df = (self.df[lambda x: x[self.config[column_name]]
                                  .duplicated(keep=False)]
                          .groupby(self.config[column_name])[self.config['problem_column']]
                            .agg(
                                nunique=lambda x: x.nunique(dropna=False),
                                values=lambda x: ','.join(set(val for val in x if pd.notnull(val))),
                                contains_null=lambda x: x.isnull().any()
                            )
                          .reset_index()
                          )
                logger.warning("\n" + tabulate(dup_df, headers='keys', tablefmt='psql'))
        else:
            logger.info(f"No duplicate {self.config[column_name]} found in {column_name}.")
            unique_items = self.df[self.config[column_name]].unique().tolist()
            logger.info(f"Unique {self.config[column_name]}: {len(unique_items)}")
    
    def check_missing(self, column_name):
        logger.info(f"Config keys used: {column_name}")
        if self.df[self.config[column_name]].isna().any():
            missing_count = self.df[self.config[column_name]].isna().sum()
            total_count = len(self.df)
            ratio = missing_count / total_count if total_count > 0 else 0
            logger.warning(
                f"Missing {self.config[column_name]} found in {column_name}. "
                f"Count: {missing_count}, Ratio: {missing_count}/{total_count} = {ratio:.4f} ({ratio:.2%})"
            )
        else:
            logger.info(f"No missing {self.config[column_name]} found in {column_name}.")

    def check_priority_problems(self, priority_list, has_problem=True):
        logger.info(f"Config keys used: {priority_list}, problem_column, priority_column")
        condition = self.df[self.config['priority_column']].isin(self.config[priority_list]) & self.df[self.config['problem_column']].isna()
        if has_problem:
            condition = ~condition
        if self.df[condition].empty:
            logger.info(f"All {self.config[priority_list]} priority cases with {'no ' if not has_problem else ''}linking tickets found.")
        else:
            logger.warning(f"Some {self.config[priority_list]} priority cases with {'no ' if not has_problem else ''}linking tickets found.")
            if not has_problem:
                total = self.df[self.df[self.config['priority_column']].isin(self.config[priority_list])].shape[0]
                no_problem = self.df[condition].shape[0]
                if total > 0:
                    ratio = no_problem / total
                    logger.info(
                        f"Ratio of {self.config[priority_list]} priority cases with no linking tickets: "
                        f"{no_problem}/{total} = {ratio:.2%} "
                        f"({no_problem} cases out of {total})"
                    )
                else:
                    logger.info(f"No {self.config[priority_list]} priority cases found in created cases.")

    def clean_str_column(self, col_name):
        logger.info(f"Config keys used: string_columns:{col_name}")
        self.df = (
            self.df
            .assign(**{col_name: lambda x: x[col_name].str.upper().str.strip()})
            .assign(**{col_name: lambda x: x[col_name].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')})
            .assign(**{col_name: lambda x: x[col_name].str.replace(r'[^A-Z0-9_]', '_', regex=True)})
            .assign(**{col_name: lambda x: x[col_name].str.replace(r'_+', '_', regex=True)})
            .assign(**{col_name: lambda x: x[col_name].str.replace(r'_$', '', regex=True)})
        )

    def get_pivot(self, index_columns, columns=None, df=None):
        
        def changes(pivot, percentage=10, min_change_units=10):
            pivot = pivot.iloc[:, :-1]  # Eliminar la columna TOTAL

            # Calcular cambios porcentuales sin rellenar NA automáticamente
            changes = pivot.pct_change(axis=1, fill_method=None) * 100

            # Crear una lista para almacenar los cambios más abruptos
            abrupt_changes = []

            # Recopilar información sobre los cambios más grandes
            for cause in pivot.index:
                # analiza si es un index simple o si viene un index tupla. Si es index sencillo seran cambios de año de 2023 a 2025, si es tupla primero esta agrupado por un string(categoria) y luego por el año
                # como en unos años puede desaparecer ya que no hubieron casos en esa categoria entonces no se podra hacer comparacion. 
                for i in range(1, len(pivot.columns)):
                    current_period = pivot.columns[i]
                    previous_period = pivot.columns[i - 1]

                    change = None  # Inicializar la variable change

                    # Determinar si las columnas son tuplas o simples años
                    if isinstance(current_period, tuple) and isinstance(previous_period, tuple):
                        # Comparar si el primer elemento es el mismo y el segundo es diferente (para tuplas)
                        if current_period[0] == previous_period[0] and current_period[1] > previous_period[1]:
                            change = changes.loc[cause, current_period]
                    elif isinstance(current_period, int) and isinstance(previous_period, int):
                        # Comparar si los años son diferentes (para años simples)
                        if current_period != previous_period:
                            change = changes.loc[cause, current_period]

                    # Solo proceder si change ha sido asignado, es positivo, y el cambio en unidades es mayor al umbral
                    if change is not None and pd.notna(change) and change > 0:
                        previous_value = pivot.loc[cause, previous_period]
                        current_value = pivot.loc[cause, current_period]
                        if pd.notna(previous_value) and pd.notna(current_value) and (current_value - previous_value) > min_change_units:
                            abrupt_changes.append((cause, previous_period, current_period, change, previous_value, current_value))
                            
            # Verificar si hay cambios abruptos antes de proceder
            if not abrupt_changes:
                logger.info("No se encontraron cambios abruptos que cumplan con los criterios especificados.")
                return

            # Ordenar los cambios por magnitud
            abrupt_changes.sort(key=lambda x: abs(x[3]), reverse=True)

            # Mostrar los 10 cambios más grandes
            num_changes_to_show = min(10, len(abrupt_changes))

            # Mostrar los cambios más abruptos
            max_cause_length = max(len(str(cause)) for cause, _, _, _, _, _ in abrupt_changes)
            for cause, previous_period, current_period, change, previous_value, current_value in abrupt_changes[:num_changes_to_show]:
                previous_period_str = f"{previous_period}" if isinstance(previous_period, int) else f"{previous_period[0]}, {previous_period[1]}"
                current_period_str = f"{current_period}" if isinstance(current_period, int) else f"{current_period[0]}, {current_period[1]}"
                cause_str = str(cause)
                logger.warning(f"Para '{cause_str:<{max_cause_length}}', el cambio más grande fue de {change:>8.2f}% (+{current_value - previous_value:>4}) pasando de {previous_value:>4} a {current_value:>4} entre el período {previous_period_str} a {current_period_str}.")

        
        logger.info(f"Config keys used: {index_columns}, {columns if columns else 'pivot_columns'}, number_column")

        if df is None:
            df = self.df

        index = [self.config[col] for col in index_columns]

        if columns is None:
            columns = self.config['pivot_columns']
        else:
            columns = [self.config[col] for col in columns]

        pivot = (
            df
            .pivot_table(
                index=index,
                columns=columns,
                values=self.config['number_column'],
                aggfunc='count',
                fill_value=0
            )
        )
        multi_index = pivot.columns.nlevels > 1
        if multi_index:
            pivot = flatten_df_columns(pivot)

        pivot['TOTAL'] = pivot.sum(axis=1)
        # filter pivot TOTAL != 0
        pivot = pivot[pivot['TOTAL'] != 0]
        sort_cols = ['TOTAL'] + [col for col in pivot.columns if col != 'TOTAL']
        top_recurring_cases = (
            pivot
            .sort_values(by=sort_cols, ascending=[False] + [False] * (len(sort_cols) - 1))
            .loc[:, lambda x: (x != 0).any(axis=0)]
        )
        logger.info(f"Recurring cases found in created tickets:\nIndex columns: {index}, Len: {top_recurring_cases.shape[0]}")
        try:
            logger.info("\n" + tabulate(top_recurring_cases.head(10), headers='keys', tablefmt='psql'))
            changes(pivot)
            
        except ImportError:
            logger.info('\n' + str(top_recurring_cases))

    def analyze(self):
        # Check for duplicates
        self.check_duplicates(column_name='number_column')
        self.check_duplicates(column_name='problem_column')
        # Check for missing values
        self.check_missing(column_name='number_column')
        self.check_missing(column_name='problem_column')
        # Check for priority problems
        self.check_priority_problems(priority_list='high_priority_list')
        self.check_priority_problems(priority_list='medium_priority_list', has_problem=False)
        self.check_priority_problems(priority_list='low_priority_list', has_problem=False)

        # Clean string columns
        for col in self.config['string_columns']:
            self.clean_str_column(col)

        # Generate pivot tables
        self.get_pivot(index_columns=['priority_column'], columns=['date_column'])
        self.get_pivot(index_columns=['cause_code_column'])
        self.get_pivot(index_columns=['close_code_column'])
        self.get_pivot(index_columns=['cause_code_column', 'close_code_column'])

        list_suffixes = self.config['suffixes']
        suffix_pattern = r'(' + '|'.join(list_suffixes) + r')$'

        self.get_pivot(
            index_columns=['service_offering_clean_column'],
            df=self.df.assign(SERVICE_OFFERING_clean=lambda x: x[self.config['service_offering_column']]
                            .str.replace(r'^\d+_', '', regex=True)
                            .str.replace(suffix_pattern, '', regex=True))
        )
        
        self.get_pivot(
            index_columns=['service_offering_clean_column'],
            df=self.df.assign(SERVICE_OFFERING_clean=lambda x: x[self.config['service_offering_column']]
                            .str.replace(r'^\d+_', '', regex=True)
                            .str.replace(suffix_pattern, '', regex=True)
                            .str.split('_').str[-1])
        )
        
        self.get_pivot(
            index_columns=['service_offering_clean_column'],
            df=self.df.assign(SERVICE_OFFERING_clean=lambda x: x[self.config['service_offering_column']]
                            .str.replace(r'^\d+_', '', regex=True)
                            .str.replace(suffix_pattern, '', regex=True)
                            .str.split('_').str[0])
        )

        self.get_pivot(index_columns=['business_service_column'])
        self.get_pivot(index_columns=['creator_group_column'])

        for p in self.df[self.config['priority_column']].unique():
            self.get_pivot(
                index_columns=['assignment_group_column', 'filtro_creator_group_column', 'filtro_grupo_column'],
                df=self.df.loc[lambda x: x[self.config['priority_column']] == p]
            )

        # display(self.df)

# Configuration dictionary
config = {
    'number_column': 'NUMBER',
    'problem_column': 'PROBLEM',
    # 'problem_column': 'CREATED_OUT_OF_INCIDENT',
    'priority_column': 'CURRENT_PRIORITY',
    'date_column': '_CREATED_year',
    # 'date_column': 'CREATED_year',
    # 'cause_code_column': 'CAUSE_CODE',
    'cause_code_column': 'STATE',
    'close_code_column': 'CLOSE_CODE',
    'service_offering_column': 'SERVICE_OFFERING',
    'service_offering_clean_column': 'SERVICE_OFFERING_clean',
    'business_service_column': 'BUSINESS_SERVICE',
    # 'business_service_column': 'TASK_TYPE',
    'creator_group_column': 'CREATOR_GROUP',
    'assignment_group_column': 'ASSIGNMENT_GROUP',
    # 'filtro_creator_group_column': 'FILTRO_CREATOR_GROUP',
    'filtro_creator_group_column': 'PRIORITY',
    # 'filtro_grupo_column': 'FILTRO_GRUPO',
    'filtro_grupo_column': 'AFFECTED_OES',
    'string_columns': ['CAUSE_CODE', 'CLOSE_CODE', 'SERVICE_OFFERING', 'BUSINESS_SERVICE', 'CREATOR_GROUP', 'ASSIGNMENT_GROUP'],
    # 'string_columns': ['STATE', 'CLOSE_CODE', 'SERVICE_OFFERING', 'TASK_TYPE', 'CREATOR_GROUP', 'ASSIGNMENT_GROUP', 'PRIORITY', 'AFFECTED_OES'],
    'pivot_columns': ['CURRENT_PRIORITY', '_CREATED_year'],
    # 'pivot_columns': ['CURRENT_PRIORITY', 'CREATED_year'],
    'high_priority_list': ['P2', 'P1'],
    'medium_priority_list': ['P3'],
    'low_priority_list': ['P4'],
    'suffixes': ['_ESP', '_IBL', '_PT', '_BR', '_CO', '_ES', '_US', '_AZMEX', '_IBEROLATAM']
}

# Create an instance of IncidentAnalyzer
analyzer = IncidentAnalyzer(df_incidents_grouped.pipe(create_dt_time_wvariants), config)
print(df_incidents_grouped.columns)
# Execute the analysis
analyzer.analyze()

# add column of %s and examples

In [ ]:
# import random

# df = df_incidents.copy()
# # Group by 'NUMBER' and calculate the number of unique values in 'RESOLVED' and 'CLOSED'
# df_inter = df.groupby('NUMBER')[['RESOLVED', 'CLOSED']].nunique()

# # Filter to find 'NUMBER' where either 'RESOLVED' or 'CLOSED' has more than 1 unique value
# reopened_tickets = df_inter[(df_inter['RESOLVED'] > 1) | (df_inter['CLOSED'] > 1)].index.tolist()

# # Sample 10 elements from the list of reopened tickets
# sampled_reopened_tickets = random.sample(reopened_tickets, min(5, len(reopened_tickets)))

# # print("Tickets that have been reopened:", reopened_tickets)

# cols = ['NUMBER', 'PROBLEM', 'CREATED', 'RESOLVED', 'CREATED_BY', 'PRIORITY', 'CURRENT_PRIORITY', 'CLOSED', 'UPDATED']
# df[cols][df['NUMBER'].isin(sampled_reopened_tickets)].to_dict(orient="records")

# print(df.columns)
# display(df)

# test77_reopened_incidents_TREND(df)

In [ ]:
# df_incidents_grouped.sample(5).to_dict(orient="records")
# display(df_incidents_grouped.head())
# # Example usage
# # Assuming df is your DataFrame
# result = test01_problems_aging(df_incidents_grouped)
# tabulate_df(result)

In [ ]:
# df_incidents.loc[lambda x: x['NUMBER'] == 'INC27459004'].sort_values(by=['NUMBER','REASSIGNMENT_COUNT','UPDATED']).nunique().reset_index().set_index('index')

In [ ]:
df_incidents_grouped.shape

In [ ]:

result = (pd.merge(df_incidents_grouped.add_suffix("_incident"), 
                              df_problem.add_suffix("_problem"), 
                              left_on="NUMBER_incident", 
                              right_on="CREATED_OUT_OF_INCIDENT_problem", 
                              suffixes=("_incident", "_problem"), 
                              how="outer", 
                              indicator=True, 
                              validate="m:m").rename(columns={"_merge": "_merge_inc_pro"})
                    #  [['NUMBER_incident','NUMBER_problem']].value_counts(sort=True)
                    # .NUMBER_problem.value_counts(sort=True, dropna=False) # ok 1:m
                    .merge(df_problem_tasks.add_suffix("_task"), 
                           left_on="NUMBER_problem", 
                           right_on="PROBLEM_task", 
                           suffixes=("", "_task"), 
                           how="outer", 
                           indicator=True, 
                           validate="m:m").rename(columns={"_merge": "_merge_pro_task"}) # m:m por nas
                    #  ._merge_pro_task.value_counts()
                    .sort_values(by=["CREATED_incident","RESOLVED_incident"])
                    # .to_excel(intermediate_data+"//merged_incidents_servicenow.xlsx", index=False)
)

casos_audit["INC_PBR_PBT_merged"] = result

In [ ]:
# (pd.merge(df_created_incidents, df_resolved_incidents, on='NUMBER', suffixes=('_created', '_resolved'), how='outer', indicator=True, validate="one_to_one")
#  .sort_values(by=["DATE_CREATED", "DATE_RESOLVED", "CLOSED", "CREATED", "RESOLVED"])
#  .to_excel(intermediate_data+"//merged_incidents.xlsx", index=False)
# )

In [ ]:
df_problem

In [ ]:
casos_audit["INCIDENTS_ES"] = df_incidents_grouped
casos_audit["INCIDENTS_ES_rows"] = df_incidents
casos_audit["PROBLEMS_ES"] = df_problem
casos_audit["PROBLEMS_TASKS_ES"] = df_problem_tasks

casos_audit['COLS_df_incidents_grouped'] = analyze_dataframe_columns_export(df_incidents_grouped)
casos_audit['COLS_df_problem'] = analyze_dataframe_columns_export(df_problem)
casos_audit['COLS_df_problem_tasks'] = analyze_dataframe_columns_export(df_problem_tasks)

In [ ]:
def _flatten_df(df):
    """Convierte columnas MultiIndex en nombres planos."""
    if isinstance(df.columns, pd.MultiIndex):
        df = df.copy()
        df.columns = [
            "_".join([str(c) for c in col if c]) for col in df.columns.values
        ]
    return df.reset_index()  # aseguramos que el índice no sea jerárquico


import openpyxl
from openpyxl.styles import Font
from openpyxl.utils import get_column_letter

def create_excel_with_links(df_dict_to_export, name_file, path="."):
    """
    Exporta un diccionario de DataFrames a un Excel con:
      - Hoja inicial de resumen (filas, columnas, nombres de columnas)
      - Hipervínculos desde la hoja inicial a cada sheet
      - Ajuste automático de anchos de columna
    """

    output_path = os.path.join(path, name_file)

    # Crear Excel temporal con todas las hojas
    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        # Hoja inicial con metadatos
        initial_df = pd.DataFrame({
            "Sheet Name": list(df_dict_to_export.keys()),
            "Rows": [df.shape[0] for df in df_dict_to_export.values()],
            "Columns": [df.shape[1] for df in df_dict_to_export.values()],
            "Column Names": [
                ", ".join([str(col) for col in df.columns.tolist()])
                for df in df_dict_to_export.values()
            ]
        })
        initial_df.to_excel(writer, sheet_name="Initial", index=False)

        # Escribir cada DataFrame (aplanando si hace falta)
        for sheet_name, df in df_dict_to_export.items():
            df_flat = _flatten_df(df)
            df_flat.to_excel(writer, sheet_name=sheet_name, index=False)

    # Reabrir con openpyxl para aplicar hipervínculos y estilos
    workbook = openpyxl.load_workbook(output_path)
    summary_sheet = workbook["Initial"]

    # Hipervínculos en columna A
    for row in range(2, len(df_dict_to_export) + 2):
        cell = summary_sheet.cell(row=row, column=1)
        sheet_name = cell.value
        cell.hyperlink = f"#{sheet_name}!A1"
        cell.font = Font(underline="single", color="0563C1")

    # Ajuste de anchos
    for col in range(1, summary_sheet.max_column + 1):
        column_letter = get_column_letter(col)
        max_length = max(
            len(str(summary_sheet.cell(row=row, column=col).value))
            for row in range(1, summary_sheet.max_row + 1)
        )
        summary_sheet.column_dimensions[column_letter].width = min(max_length + 3, 100)

    # Guardar final
    workbook.save(output_path)
    print(f"✅ Excel con hipervínculos creado en: {output_path}")

# Specify the file name for the Excel file
excel_file_name = 'DA_INCIDENT_PROBLEMS.xlsx'
create_excel_with_links(casos_audit, excel_file_name, path=".")